In [9]:
import pandas as pd
vanila_dataset = pd.read_csv("/opt/ml/dataset/train/dup_clean_train.csv", index_col = 0, encoding = 'utf-8')
vanila_dataset['id'] = [i for i in range(len(vanila_dataset))]
vanila_dataset.reset_index(drop=False, inplace=True)
vanila_dataset

data= vanila_dataset

sub_entity, sub_type= [], []
obj_entity, obj_type= [], []
sub_idx, obj_idx= [], []
sentence= []

"""preprocess"""
for i, [x, y, z] in enumerate(zip(data['subject_entity'], data['object_entity'], data['sentence'])):
    sub_typ= x[1:-1].split(':')[-1].split('\'')[-2]
    obj_typ= y[1:-1].split(':')[-1].split('\'')[-2]
            
    for idx_i in range(len(x)):
        if x[idx_i: idx_i+ 9]== 'start_idx':
            sub_start= int(x[idx_i+12:].split(',')[0].strip())
        if x[idx_i: idx_i+7]== 'end_idx':
            sub_end= int(x[idx_i+10:].split(',')[0].strip())
                
        if y[idx_i: idx_i+ 9]== 'start_idx':
            obj_start= int(y[idx_i+12:].split(',')[0].strip())
        if y[idx_i: idx_i+7]== 'end_idx':
            obj_end= int(y[idx_i+10:].split(',')[0].strip())

    sub_i= [sub_start, sub_end]
    obj_i= [obj_start, obj_end]

    sub_entity.append(z[sub_i[0]: sub_i[1]+1])
    obj_entity.append(z[obj_i[0]: obj_i[1]+1])
    sub_type.append(sub_typ); sub_idx.append(sub_i)
    obj_type.append(obj_typ); obj_idx.append(obj_i)
        
    sentence.append(z)
df = pd.DataFrame({'id': data['id'], 'sentence' : sentence, 'subject_entity': sub_entity, 'object_entity': obj_entity,
                        'subject_type': sub_type, 'object_type': obj_type, 'label': data['label'],
                        'subject_idx': sub_idx, 'object_idx': obj_idx})

df

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32459,32459,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32460,32460,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32461,32461,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32462,32462,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [10]:
relation_class = ['no_relation', 'org:top_members/employees', 'org:members',
        'org:product', 'per:title', 'org:alternate_names',
        'per:employee_of', 'org:place_of_headquarters', 'per:product',
        'org:number_of_employees/members', 'per:children',
        'per:place_of_residence', 'per:alternate_names',
        'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
        'per:spouse', 'org:founded', 'org:political/religious_affiliation',
        'org:member_of', 'per:parents', 'org:dissolved',
        'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
        'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
        'per:religion']

type_class = ['ORG/ORG','ORG/PER','ORG/DAT','ORG/LOC','ORG/POH','ORG/NOH',
              'PER/ORG','PER/PER','PER/DAT','PER/LOC','PER/POH','PER/NOH']

In [11]:
from tqdm.notebook import tqdm
# relation_book[relation_class][type_class][random]
relation_book = []

for _, relation_name in tqdm(enumerate(relation_class)) :
    temp_list = []
    temp_df = df[df['label'] == str(relation_name) ][['subject_entity','object_entity','subject_type','object_type', 'label']]
    
    for _ , type_name in enumerate(type_class) : 
        temp_list2 = []
        temp_df2 = temp_df[temp_df['subject_type'] + '/' + temp_df['object_type'] == str(type_name)][['subject_entity','object_entity']]
        temp_df2_idx_list = list(temp_df2.index)
        
        for i in temp_df2_idx_list : 
            temp_list3 = []
            temp_list3.append(temp_df2['subject_entity'][i])
            temp_list3.append(temp_df2['object_entity'][i])
            
            temp_list2.append(temp_list3)
        
        temp_list.append(temp_list2)
        
    relation_book.append(temp_list)

In [12]:
relation_book[27]

[[],
 [],
 [],
 [],
 [],
 [],
 [['김창익', '캐나다 벤쿠버']],
 [['이스마일 2세', '카즈빈']],
 [['진시황', '진나라']],
 [['최진영', '서울특별시 강남구'],
  ['막시밀리아노 1세', '멕시코 케레타로'],
  ['우범선', '일본 히로시마현 구레'],
  ['마리 드 메디시스', '쾰른'],
  ['김립', '상하이'],
  ['트라야누스', '세리누스'],
  ['루이스 몰로니', '라스 팔마스'],
  ['고용희', '프랑스'],
  ['레흐 카친스키', '러시아 스몰렌스크'],
  ['박정희', '서울'],
  ['알폰소 10세', '세비야'],
  ['루돌프 1세', '슈파이어'],
  ['피천득', '서울'],
  ['미셸 드브레', '몽루이쉬르루아르'],
  ['로빈 윌리엄스', '캘리포니아'],
  ['메이레키', '에도'],
  ['요한 바오로 1세', '바티칸'],
  ['서재필', '펜실베이니아주 필라델피아'],
  ['손원일', '대한민국'],
  ['헤디 라마르', '플로리다'],
  ['체 게바라', '볼리비아'],
  ['손다이크', '미국 뉴욕주'],
  ['박시창', '서울'],
  ['워버그', '뉴욕'],
  ['플로리야나 이스마일리', '이탈리아 롬바르디아주 코모호'],
  ['리승엽', '조선민주주의인민공화국'],
  ['대칸 뭉케', '중국'],
  ['미하일 아센 1세', '터르노보 근교'],
  ['여운형', '서울'],
  ['고용희', '프랑스'],
  ['브루투스', '필리피'],
  ['고드프루아 드 부용', '예루살렘'],
  ['바실리 4세', '고스티닌 성'],
  ['알렉산다르 1세', '프랑스 마르세유'],
  ['만네르헤임', '스위스 로잔']],
 [['선의왕후', '경희궁 어조당(魚藻堂)'], ['고종후', '진주성']],
 []]

In [274]:
df

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32459,32459,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32460,32460,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32461,32461,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32462,32462,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [275]:
modified_df.head(10)

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 마이크 시노다이 쓰고 린킨 파크가 1969년 앨범 《Abbe...,린킨 파크,마이크 시노다,ORG,PER,no_relation,"[25, 29]","[13, 19]"
1,1,호남이 기반인 바른미래당·경기도·양주시이 우여곡절 끝에 합당해 민생당(가칭)으로 재...,양주시,경기도,ORG,ORG,no_relation,"[18, 20]","[14, 16]"
2,2,K리그2에서 성적 1위를 달리고 있는 SS 라치오는 지난 26일 세리에 A으로부터 ...,SS 라치오,세리에 A,ORG,ORG,org:member_of,"[21, 26]","[36, 40]"
3,3,균일가 생활용품점 (주)정읍시(대표 유진섭)는 코로나19 바이러스로 어려움을 겪고 ...,정읍시,유진섭,ORG,PER,org:top_members/employees,"[13, 15]","[20, 22]"
4,4,2016년 프로 야구 드래프트 1순위로 워싱턴 내셔널스에게 입단하면서 등번호는 8번...,워싱턴 내셔널스,2016,ORG,DAT,no_relation,"[22, 29]","[0, 3]"
5,5,": 유엔, 유럽 의회, 조소카베 가문 (長宗我部氏), 국제이주기구, 세계 보건 기구...",조소카베 가문,長宗我部氏,ORG,ORG,org:alternate_names,"[13, 19]","[22, 26]"
6,6,그에 따라 나폴리와 계약을 연장한 마라도나는 2011년 팀을 UEFA컵 정상으로 인...,맨체스터 유나이티드 FC,2011,ORG,DAT,no_relation,"[64, 76]","[25, 28]"
7,7,"지승민(朴容旿, 1978년 9월 12일(음력 3월 19일)(음력 3월 19일) ~ ...",지승민,1978년 9월 12일,PER,DAT,per:date_of_birth,"[0, 2]","[9, 20]"
8,8,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 키예프로 수도를 옮기는 것...,우크라이나,키예프,ORG,LOC,org:place_of_headquarters,"[58, 62]","[32, 34]"
9,9,"특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, 강윤구 국회의원, 김종민 국회...",강윤구,넥센 히어로즈,PER,ORG,per:employee_of,"[30, 32]","[100, 106]"


In [276]:
# label, subject_type, object_type이 모두 같아야만 치환 가능
# 문장 데이터 교체 자동화가 필요함!!
import random
from tqdm.notebook import tqdm
import copy
modified_df = copy.copy(df)

for i in tqdm(range(len(df))) : 
    temp_list = []
    relation_num = relation_class.index(df['label'][i])
    type_num = type_class.index((df['subject_type'][i]+'/' + df['object_type'][i]))
    
    temp_list = relation_book[relation_num][type_num]
    random_entity = temp_list[random.randint(0, len(temp_list)-1)]
    
    # index 설정
    sub_idx = df['subject_idx'][i]    
    obj_idx = df['object_idx'][i]
    
    # sub가 먼저 나올 경우
    if min(sub_idx[0],obj_idx[0]) == sub_idx[0] :
        temp_len = len(df['sentence'][i][sub_idx[1]+1:obj_idx[0]])
        
        modified_sentence = df['sentence'][i][:sub_idx[0]] + random_entity[0] + df['sentence'][i][sub_idx[1]+1:obj_idx[0]] + random_entity[1] + df['sentence'][i][obj_idx[1]+1:]
        
        modified_df['subject_idx'][i][0] = len(df['sentence'][i][:sub_idx[0]])
        modified_df['subject_idx'][i][1] = len(df['sentence'][i][:sub_idx[0]] + random_entity[0]) -1
        modified_df['object_idx'][i][0] = modified_df['subject_idx'][i][1] + temp_len + 1
        modified_df['object_idx'][i][1] = modified_df['object_idx'][i][0] + len(random_entity[1]) -1
    
    # obj가 먼저 나올 경우
    else :
        temp_len = len(df['sentence'][i][obj_idx[1]+1:sub_idx[0]])
        
        modified_sentence = df['sentence'][i][:obj_idx[0]] + random_entity[1] + df['sentence'][i][obj_idx[1]+1:sub_idx[0]] + random_entity[0] + df['sentence'][i][sub_idx[1]+1:]
        
        modified_df['object_idx'][i][0] = len(df['sentence'][i][:obj_idx[0]]) 
        modified_df['object_idx'][i][1] = len(df['sentence'][i][:obj_idx[0]] + random_entity[1]) -1
        modified_df['subject_idx'][i][0] = modified_df['object_idx'][i][1] + temp_len + 1
        modified_df['subject_idx'][i][1] = modified_df['subject_idx'][i][0] + len(random_entity[0]) -1
        
        
    # 갈아끼우기
    modified_df['sentence'][i] = modified_sentence
    modified_df['subject_entity'][i] = random_entity[0]
    modified_df['object_entity'][i] = random_entity[1]
    
    # print(random_entity)
    # print("BEFORE : ", df['sentence'][i])
    # print("AFTER : ", modified_sentence)
    # print("\n")
modified_df

<ipython-input-276-c09fe496017f>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df['sentence'][i] = modified_sentence
<ipython-input-276-c09fe496017f>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df['subject_entity'][i] = random_entity[0]
<ipython-input-276-c09fe496017f>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df['object_entity'][i] = random_entity[1]


,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 문승현이 쓰고 광주과학기술원가 1969년 앨범 《Abbey ...,광주과학기술원,문승현,ORG,PER,no_relation,"[21, 27]","[13, 15]"
1,1,호남이 기반인 바른미래당·세리에 C·세리에 B이 우여곡절 끝에 합당해 민생당(가칭)...,세리에 B,세리에 C,ORG,ORG,no_relation,"[20, 24]","[14, 18]"
2,2,K리그2에서 성적 1위를 달리고 있는 광주 상무 축구단는 지난 26일 K리그으로부터...,광주 상무 축구단,K리그,ORG,ORG,org:member_of,"[21, 29]","[39, 41]"
3,3,균일가 생활용품점 (주)국토교통부(대표 박선호)는 코로나19 바이러스로 어려움을 겪...,국토교통부,박선호,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,2015년년 프로 야구 드래프트 1순위로 SK 와이번스에게 입단하면서 등번호는 8번...,SK 와이번스,2015년,ORG,DAT,no_relation,"[23, 29]","[0, 4]"
...,...,...,...,...,...,...,...,...,...
32459,32459,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,나루히토,일본 황실,PER,LOC,per:employee_of,"[93, 96]","[101, 105]"
32460,32460,"법포는 다시 존 아담스, 서병학, 토머스 제퍼슨 직계인 북접과 다시 서장옥, 전봉준...",존 아담스,토머스 제퍼슨,PER,PER,per:colleagues,"[7, 11]","[19, 25]"
32461,32461,화순군(군수 구충곤)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,화순군,구충곤,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32462,32462,"중앙일보, SK브로드밴드 회장을 지낸 이후 SK텔레콤 회장, 재단법인 한반도평화만들...",SK브로드밴드,SK텔레콤,ORG,ORG,no_relation,"[6, 12]","[24, 28]"


In [222]:
num = 1


sub_i = modified_df['subject_idx'][num]    
obj_i = modified_df['object_idx'][num]
    
sub_typ = modified_df['subject_type'][num]
obj_typ = modified_df['object_type'][num]
    
z = modified_df['sentence'][num]

if sub_i[0] < obj_i[0]:
    z= z[:sub_i[0]] + '@*'+sub_typ+'*'+ z[sub_i[0]: sub_i[1]+1] + '@' + z[sub_i[1]+1:]
    z= z[:obj_i[0]+7] + '#^'+ obj_typ +'^'+ z[obj_i[0]+7: obj_i[1]+8]+ '#'+ z[obj_i[1]+8:]
else:
    z= z[:obj_i[0]] + '#^'+ obj_typ +'^'+ z[obj_i[0]: obj_i[1]+1]+ '#' + z[obj_i[1]+1:]
    z= z[:sub_i[0]+7] + '@*'+sub_typ+'*' + z[sub_i[0]+7: sub_i[1]+8] + '@' + z[sub_i[1]+8:]
    
print(sub_i)
print(obj_i)
print(modified_df['sentence'][num])
print(z)


[22, 27]
[14, 20]
호남이 기반인 바른미래당·산업통상자원부·한국전력공사이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
호남이 기반인 바른미래당·#^ORG^산업통상자원부#·@*ORG*한국전력공사@이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.


In [256]:
modified_df_temp = df.copy()

In [257]:
from tqdm.notebook import tqdm

for i in tqdm(range(len(modified_df_temp))) :
    
    sub_i = modified_df_temp['subject_idx'][i]    
    obj_i = modified_df_temp['object_idx'][i]
    
    sub_typ = modified_df_temp['subject_type'][i]
    obj_typ = modified_df_temp['object_type'][i]
    
    z = modified_df_temp['sentence'][i]

    if sub_i[0] < obj_i[0]:
        z= z[:sub_i[0]] + '@*'+sub_typ+'*'+ z[sub_i[0]: sub_i[1]+1] + '@' + z[sub_i[1]+1:]
        z= z[:obj_i[0]+7] + '#^'+ obj_typ +'^'+ z[obj_i[0]+7: obj_i[1]+8]+ '#'+ z[obj_i[1]+8:]
    else:
        z= z[:obj_i[0]] + '#^'+ obj_typ +'^'+ z[obj_i[0]: obj_i[1]+1]+ '#' + z[obj_i[1]+1:]
        z= z[:sub_i[0]+7] + '@*'+sub_typ+'*' + z[sub_i[0]+7: sub_i[1]+8] + '@' + z[sub_i[1]+8:]
        
    modified_df_temp['sentence'][i] = z
modified_df_temp

<ipython-input-257-fc18b8f13bf0>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df_temp['sentence'][i] = z


,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 196...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·#^ORG^대안신당#·@*ORG*민주평화당@이 우여곡절 끝...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 @*ORG*광주FC@는 지난 26일 #^ORG...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)@*ORG*아성다이소@(대표 #^PER^박정부#)는 코로나1...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,#^DAT^1967#년 프로 야구 드래프트 1순위로 @*ORG*요미우리 자이언츠@에...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32459,32459,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32460,32460,"법포는 다시 @*PER*최시형@, 서병학, #^PER^손병희# 직계인 북접과 다시 ...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32461,32461,@*ORG*완도군@(군수 #^PER^신우철#)이 국토교통부에서 실시한 '2019 교...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32462,32462,"중앙일보, @*ORG*JTBC@ 회장을 지낸 이후 #^ORG^중앙홀딩스# 회장, 재...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [258]:
# modified_df_temp.to_csv("/opt/ml/dataset/train/orgin_train.csv", encoding='utf-8')

In [259]:
org_df1 = pd.read_csv("/opt/ml/dataset/train/orgin_train.csv", encoding = 'utf-8')
fake_df1 = pd.read_csv("/opt/ml/dataset/train/fake_train1.csv", encoding = 'utf-8')
fake_df2 = pd.read_csv("/opt/ml/dataset/train/fake_train2.csv", encoding = 'utf-8')

In [263]:
dataset_concat = pd.concat([org_df1, fake_df1, fake_df2], ignore_index=True)
dataset_concat.drop(['Unnamed: 0'], axis=1, inplace=True)
dataset_concat['id'] = [i for i in range(len(dataset_concat))]
dataset_concat

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 196...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·#^ORG^대안신당#·@*ORG*민주평화당@이 우여곡절 끝...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 @*ORG*광주FC@는 지난 26일 #^ORG...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)@*ORG*아성다이소@(대표 #^PER^박정부#)는 코로나1...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,#^DAT^1967#년 프로 야구 드래프트 1순위로 @*ORG*요미우리 자이언츠@에...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
97387,97387,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,앙리 크리스토프,아이티,PER,LOC,per:employee_of,"[93, 100]","[105, 107]"
97388,97388,"법포는 다시 @*PER*윤창중@, 서병학, #^PER^박근혜# 직계인 북접과 다시 ...",윤창중,박근혜,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
97389,97389,@*ORG*해남군@(군수 #^PER^명현관#)이 국토교통부에서 실시한 '2019 교...,해남군,명현관,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
97390,97390,"중앙일보, @*ORG*도로교통공단@ 회장을 지낸 이후 #^ORG^부산경찰청# 회장,...",도로교통공단,부산경찰청,ORG,ORG,no_relation,"[6, 11]","[23, 27]"


In [267]:
# 한번 섞고 인덱스 다시
df_shuffled=dataset_concat.sample(frac=1).reset_index(drop=True)
df_shuffled['id'] = [i for i in range(len(dataset_concat))]
df_shuffled

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,@*PER*이용섭@ 광주광역시장은 22일 오전 국회에서 열린 #^ORG^더불어민주당...,이용섭,더불어민주당,PER,ORG,per:employee_of,"[0, 2]","[27, 32]"
1,1,#^DAT^1990년#년에 리그 우승과 컵 대회 우승을 동시에 하는 더블을 기록하였...,KIA 타이거즈,1990년,ORG,DAT,no_relation,"[74, 81]","[0, 4]"
2,2,GC@*ORG*녹십자@(대표 #^PER^허은철#)는 임직원간 소통 확대를 위해 7월...,녹십자,허은철,ORG,PER,org:top_members/employees,"[2, 4]","[9, 11]"
3,3,"우승자는 박재범(JAYPARK), 사이먼 도미닉(SIMON DOMINIC), 그레이...",광주지방경찰청,전남지방경찰청,ORG,ORG,no_relation,"[47, 53]","[98, 104]"
4,4,"#^DAT^2015년 12월 29일# 왕이 훙(薨)하니, 호를 @*PER*김양건@(...",김양건,2015년 12월 29일,PER,DAT,per:date_of_death,"[28, 30]","[0, 12]"
...,...,...,...,...,...,...,...,...,...
97387,97387,#^PER^아스트로#는 최근 건강 문제로 활동을 중단한 문빈을 제외한 5인 체제(@...,차은우,아스트로,PER,PER,per:employee_of,"[38, 40]","[0, 3]"
97388,97388,"업무협약에는 이승주 관산읍장, #^PER^이희진# @*ORG*영덕군@교총장, 강경구...",영덕군,이희진,ORG,PER,org:top_members/employees,"[21, 23]","[17, 19]"
97389,97389,@*ORG*유니세프@ 공익법인인 #^ORG^한국위원회#은 여성가족부 산하 한국건강가...,유니세프,한국위원회,ORG,ORG,org:members,"[0, 3]","[11, 15]"
97390,97390,@*PER*네빌 롱바텀@ #^ORG^호그와트# 의원은 산은 인베스트먼트 출범을 문제...,네빌 롱바텀,호그와트,PER,ORG,per:employee_of,"[0, 5]","[7, 10]"


In [268]:
dataset_concat.to_csv("/opt/ml/dataset/train/fake_train_concat.csv", encoding='utf-8')

1. 실패

기존 데이터
+
entity 바꾼것 1
+
entity 바꾼것 2


= 10만

In [ ]:
2. 

In [6]:
vanila_dataset['sentence'][0]

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

3. entiy 랜덤, 문장도 랜덤

조지 해리슨이 쓰고 비틀즈 per/ org no_relation

|문승현|이 쓰고 |과학기술원|
